In [3]:
from sparknlp.annotator import *
import transformers
from sparknlp.base import *
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .appName("Spark NLP") \
      .master("local[*]") \
      .config("spark.driver.memory", "16G") \
      .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
      .config("spark.kryoserializer.buffer.max", "2000M") \
      .config("spark.driver.maxResultSize", "0") \
      .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.3.3") \
      .config("spark.jars", "/shareddata/lab09/kafka-clients-3.5.0.jar,/shareddata/lab09/spark-sql-kafka-0-10_2.12-3.5.0.jar,/shareddata/lab09/spark-token-provider-kafka-0-10_2.12-3.5.0.jar,/shareddata/lab09/commons-pool2-2.12.0.jar") \
      .getOrCreate()

:: loading settings :: url = jar:file:/opt/module/spark-3.5.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4e1fe3bd-a122-4400-91bf-0761884b9d35;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.3.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central
	found com.g

In [6]:
text = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "localhost:64046").option("subscribe", "qa").option("startingOffsets", "earliest").load()
text.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [11]:
MODEL_NAME = "/data/lab/Projects/Project2/best_model"
EXPORT_PATH = f"""./best_model/onnx_models/flan-t5-small"""
! optimum-cli export onnx --task text2text-generation-with-past --model {MODEL_NAME} {EXPORT_PATH}
# ! mkdir -p {EXPORT_PATH}/assets
# ! mv -t {EXPORT_PATH}/assets {EXPORT_PATH}/spiece.model

/root/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Framework not specified. Using pt to export the model.
Using the export variant default. Available variants are:
    - default: The default ONNX variant.

***** Exporting submodel 1/3: T5Stack *****
Using framework PyTorch: 2.3.0+cu121
Overriding 1 configuration item(s)
	- use_cache -> False

***** Exporting submodel 2/3: T5ForConditionalGeneration *****
Using framework PyTorch: 2.3.0+cu121
Overriding 1 configuration item(s)
	- use_cache -> True
/root/anaconda3/lib/python3.11/site-packages/trans

In [12]:
T5 = T5Transformer.loadSavedModel(EXPORT_PATH, spark)\
   .setUseCache(True) \
   .setTask("qa:") \
   .setMaxOutputLength(200)

T5.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")
# !rm -rf {EXPORT_PATH}

IllegalArgumentException: requirement failed: File spiece.model not found in folder /data/lab/Projects/Project2/./best_model/onnx_models/flan-t5-small/assets

In [4]:
test_data = spark.createDataFrame([
    ["Transfer learning, where a model is first pre-trained on a data-rich task before being fine-tuned on a " +
       "downstream task, has emerged as a powerful technique in natural language processing (NLP). The effectiveness" +
       " of transfer learning has given rise to a diversity of approaches, methodology, and practice. In this " +
       "paper, we explore the landscape of transfer learning techniques for NLP by introducing a unified framework " +
       "that converts all text-based language problems into a text-to-text format. Our systematic study compares " +
       "pre-training objectives, architectures, unlabeled data sets, transfer approaches, and other factors on dozens " +
       "of language understanding tasks. By combining the insights from our exploration with scale and our new " +
       "Colossal Clean Crawled Corpus, we achieve state-of-the-art results on many benchmarks covering " +
       "summarization, question answering, text classification, and more. To facilitate future work on transfer " +
       "learning for NLP, we release our data set, pre-trained models, and code."]
]).toDF("text")


document_assembler = DocumentAssembler() \
    .setInputCol("text")\
    .setOutputCol("document")

T5 = T5Transformer.load(f"{MODEL_NAME}_spark_nlp") \
  .setInputCols(["document"]) \
  .setOutputCol("summary")

pipeline = Pipeline().setStages([document_assembler, T5])

result = pipeline.fit(test_data).transform(test_data)
result.select("summary.result").show(truncate=False)

Using CPUs
Using CPUs


+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                          |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[We explore the landscape of transfer learning techniques for NLP by introducing a unified framework that converts all text-based language problems into a text-to-text format.]|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

